In [ ]:
#load the packages
import numpy as np
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pandas as pd
import json
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision.transforms as transforms
from collections import Counter
from pathlib import Path
from io import BytesIO
from torch.utils.data import Dataset, DataLoader

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import json
from tqdm import tqdm

# ============================================================
#                 CONFIGURATION
# ============================================================
CSV_PATH = "all_games.csv"  # Your dataset

LIST_OF_PLAYERS = [
    "Carlsen, Magnus",
    "Firouzja, Alireza",
    "Caruana, Fabiano",
    "Nepomniachtchi, Ian",
    "Cramling Bellon, Anna",
    "Giri, Anish",
    "Niemann, Hans Moke",
    "Cramling, Pia",
    "Nakamura, Hikaru",
    "Botez, Alexandra",
    "Botez, Andrea",
    "Belenkaya, Dina",
    "So, Wesley",
]

# ---- Only use FIRST 4 players ----
PLAYERS = LIST_OF_PLAYERS[:4]

MAX_LEN = 60
BATCH_SIZE = 256
EMB_DIM = 128
HIDDEN_DIM = 128
EPOCHS = 30
LR = 2e-3

# ============================================================
#                 LOAD CSV
# ============================================================
print("Loading CSV...")
df = pd.read_csv(CSV_PATH)

df["white_name"] = df["white_name"].astype(str)
df["black_name"] = df["black_name"].astype(str)

# ============================================================
#                 LABEL CONSTRUCTION (5 players only)
# ============================================================
def extract_player(name):
    for p in PLAYERS:
        if p.lower() in name.lower():
            return p
    return None

df["label"] = df.apply(
    lambda row: extract_player(row["white_name"]) or extract_player(row["black_name"]),
    axis=1,
)

# Keep ONLY games involving the first 5 players
df = df[df["label"].notna()].reset_index(drop=True)

print("Games left after filtering:", len(df))
print(df["label"].value_counts())

# ============================================================
#                 MOVE TOKENIZATION
# ============================================================
print("Building vocabulary...")

all_moves = []
for moves in df["list_of_moves"]:
    moves = moves.strip("[]").replace("'", "").replace(",", "")
    tokens = moves.split()
    all_moves.extend(tokens)

move_counts = Counter(all_moves)

vocab = {"<PAD>": 0, "<UNK>": 1}
for move in move_counts:
    vocab[move] = len(vocab)

with open("vocab.json", "w") as f:
    json.dump(vocab, f)

# ============================================================
#                 DATASET CLASS
# ============================================================
label2id = {name: i for i, name in enumerate(PLAYERS)}
id2label = {i: name for name, i in label2id.items()}

def encode_moves(moves):
    tokens = moves.strip("[]").replace("'", "").replace(",", "").split()
    idxs = [vocab.get(t, 1) for t in tokens][:MAX_LEN]
    while len(idxs) < MAX_LEN:
        idxs.append(0)
    return idxs

class ChessDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        moves = encode_moves(row["list_of_moves"])
        label = label2id[row["label"]]
        return torch.tensor(moves, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# ============================================================
#                 TRAIN/VAL/TEST SPLIT
# ============================================================
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42, shuffle=True)

train_loader = DataLoader(ChessDataset(train_df), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(ChessDataset(val_df),   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(ChessDataset(test_df),  batch_size=BATCH_SIZE, shuffle=False)

# ============================================================
#                 MODEL
# ============================================================
class ChessRNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        _, h = self.rnn(x)
        h = self.dropout(h.squeeze(0))
        return self.fc(h)

model = ChessRNN(len(vocab), EMB_DIM, HIDDEN_DIM, len(label2id))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ============================================================
#                 TRAINING LOOP
# ============================================================
print("Training model...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X, y in tqdm(train_loader, colour='green'):
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += len(y)

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for Xv, yv in val_loader:
            preds = model(Xv).argmax(dim=1)
            val_correct += (preds == yv).sum().item()
            val_total += len(yv)

    val_acc = val_correct / val_total

    print(
        f"Epoch {epoch+1}/{EPOCHS} "
        f"Loss = {total_loss:.3f}  "
        f"Train Acc = {train_acc:.3f}  "
        f"Val Acc = {val_acc:.3f}"
    )

# ============================================================
#                 FINAL TEST ACCURACY
# ============================================================
print("Evaluating test accuracy...")

model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for Xt, yt in test_loader:
        preds = model(Xt).argmax(dim=1)
        test_correct += (preds == yt).sum().item()
        test_total += len(yt)

print(f"FINAL TEST ACCURACY = {test_correct / test_total:.3f}")

# ============================================================
#                 SAVE MODEL
# ============================================================
torch.save(model.state_dict(), "rnn_player_classifier.pt")
print("Model saved as rnn_player_classifier.pt")


Loading CSV...
Games left after filtering: 24348
label
Carlsen, Magnus        7726
Caruana, Fabiano       6418
Nepomniachtchi, Ian    5157
Firouzja, Alireza      5047
Name: count, dtype: int64
Building vocabulary...
Training model...


100%|██████████| 67/67 [00:12<00:00,  5.56it/s]


Epoch 1/30 Loss = 92.551  Train Acc = 0.300  Val Acc = 0.315


100%|██████████| 67/67 [00:11<00:00,  5.72it/s]


Epoch 2/30 Loss = 90.312  Train Acc = 0.349  Val Acc = 0.320


...

Epoch 28/30 Loss = 3.969  Train Acc = 0.981  Val Acc = 0.382


100%|██████████| 67/67 [00:20<00:00,  3.32it/s]


Epoch 29/30 Loss = 4.203  Train Acc = 0.980  Val Acc = 0.382


100%|██████████| 67/67 [00:16<00:00,  4.05it/s]


Epoch 30/30 Loss = 3.235  Train Acc = 0.984  Val Acc = 0.386
Evaluating test accuracy...
FINAL TEST ACCURACY = 0.400
Model saved as rnn_player_classifier.pt
